<center><h1>Google AI Assistant for Data Science teaching</h1></center>

<center><img src="https://res.infoq.com/news/2024/02/google-gemma-open-model/en/headerimage/generatedHeaderImage-1708977571481.jpg" width="400"></center>

# Introduction

This Notebook will build an AI Assistant for Data Science teaching.

## The method

We will fine-tune a LLM with Data Science questions and answers. Then we will create a custom class that will query this model.

## The model

As model, we will use Gemma model, fine-tuned with our Data Science Q&A data, using LoRA.

## The data

As data for fine-tuning Gemma, we will use [Data Science Q&A Treasury](https://www.kaggle.com/datasets/memocan/data-science-interview-q-and-a-treasury) dataset. This dataset contains over 150 questions and answering about Data Science.

## Previous work

This work is largely based on previous work. Here I list the sources:

1. Gemma Model Card, Kaggle Models, https://www.kaggle.com/models/google/gemma
2. Kaggle QA with Gemma - KerasNLP Starter, Kaggle Code, https://www.kaggle.com/code/awsaf49/kaggle-qa-with-gemma-kerasnlp-starter (Version 11)
3. Fine-tune Gemma models in Keras using LoRA, Kaggle Code, https://www.kaggle.com/code/nilaychauhan/fine-tune-gemma-models-in-keras-using-lora (Version 1)
4. Edward J. Hu, Yelong Shen, Phillip Wallis, Zeyuan Allen-Zhu, Yuanzhi Li, Shean Wang, Lu Wang, Weizhu Chen, LoRA: Low-Rank Adaptation of Large Language Models, ArXiv, https://arxiv.org/pdf/2106.09685.pdf
5. Abheesht Sharma, Matthew Watson, Parameter-efficient fine-tuning of GPT-2 with LoRA, https://keras.io/examples/nlp/parameter_efficient_finetuning_of_gpt2_with_lora/
6. Keras 3 API documentation / KerasNLP / Models / Gemma, https://keras.io/api/keras_nlp/models/gemma/   
7. Understanding LoRA with a minimal example, https://blogs.rstudio.com/ai/posts/2023-06-22-understanding-lora/


# Introduction about Gemma


Gemma is a collection of lightweight source generative AI models designed to be used mostly by developers and researchers. Created by Google DeepMind research lab that also developed Gemini, Gemma is available in several versions, with 2B and 7B parameters, as following:

| Model                  | Parameters      | Tuned versions    | Description                                    | Recomemnded target platforms       |
|------------------------|-----------------|-------------------|------------------------------------------------|------------------------------------|
| `gemma_2b_en`          | 2.51B           | Pretrained        | 18-layer Gemma model (Gemma with 2B parameters)|Mobile devices and laptops          |
| `gemma_instruct_2b_en` | 2.51B           | Instruction tuned | 18-layer Gemma model (Gemma with 2B parameters)| Mobile devices and laptops         | 
| `gemma_7b_en`          | 8.54B           | Pretrained        | 28-layer Gemma model (Gemma with 7B parameters)| Desktop computers and small servers|
| `gemma_instruct_7b_en` | 8.54B           | Instruction tuned | 28-layer Gemma model (Gemma with 7B parameters)| Desktop computers and small servers|


For this notebook, we will fine-tune `gemma_2b_en` model, one of the `2B` parameters Gemma models.

# LoRA introduction

LoRA stands for Low-Rank Adaptation. It is a method used to fine-tune large language models (LLMs) by freezing the weights of the LLM and injecting trainable rank-decomposition matrices.   
The number of trainable parameters during fine-tunning will decrease therefore considerably.   
According to LoRA paper, this number decreases 10,000 times, and the computational resources size decreases 3 times. 

# Installations and configurations

In [1]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.1.1 which is incompatible.


In [2]:
import os
os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00" # avoid memory fragmentation on JAX backend.

import keras
import keras_nlp

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from time import time
tqdm.pandas() # progress bar for pandas

from IPython.display import display, Markdown

2024-04-07 09:27:23.212348: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 09:27:23.212447: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 09:27:23.334555: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
class Config:
    seed = 42
    dataset_path = "/kaggle/input/data-science-interview-q-and-a-treasury/dataset.csv"
    preset = "gemma_2b_en" # name of pretrained Gemma
    sequence_length = 512 # max size of input sequence for training
    batch_size = 1 # size of the input batch in training
    epochs = 10 # number of epochs to train

In [4]:
keras.utils.set_random_seed(Config.seed)

Let's define as well a small utility function for the output.

In [5]:
def colorize_text(text):
    for word, color in zip(["Question", "Answer", "Total time"], ["blue", "red" "blue"]):
        text = text.replace(f"\n\n{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

# Load the data  

We load the data we will use for fine-tunning with LoRA.

In [6]:
df = pd.read_csv(f"{Config.dataset_path}")
df.head()

,question,answer
0,What is supervised machine learning? 👶,Supervised learning is a type of machine learn...
1,What is regression? Which models can you use t...,Regression is a part of supervised ML. Regress...
2,What is linear regression? When do we use it? 👶,Linear regression is a model that assumes a li...
3,What are the main assumptions of linear regres...,There are several assumptions of linear regres...
4,What’s the normal distribution? Why do we care...,The normal distribution is a continuous probab...


# Fine-tune Gemma

We prepare a template and generate a prompt using this template from each row in the dataset.

In [7]:
template = "\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"
df["prompt"] = df.progress_apply(lambda row: template.format(question=row.question,
                                                             answer=row.answer), axis=1)
data = df.prompt.tolist()

  0%|          | 0/166 [00:00<?, ?it/s]

## Initialize the code for Gemma Causal LM

We initialize `GemmaCausalML` with `gemma_2b_en` model.

In [8]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

## Gemma preprocessor

In [9]:
x, y, sample_weight = gemma_lm.preprocessor(data[0:1])

## Enable LoRA for the model

We set LoRA rank to 5. The higher the LoRA rank, the higher the number of trainable parameters.

In [10]:
# Enable LoRA for the model and set the LoRA rank to 5.
gemma_lm.backbone.enable_lora(rank=5)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,877,376 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,877,376 (9.34 GB)

 Trainable params: 1,704,960 (6.50 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

The total trainable parameters is 1.7 M (or 6.5 MB).  
This is less than 0.06% of the 2,5G (9.35 GB) total trainable parameters.

## Run the fine-tuning sequence

In [11]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = Config.sequence_length 

# Compile the model with loss, optimizer, and metric
gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=8e-5),
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train model
gemma_lm.fit(data, epochs=Config.epochs, batch_size=Config.batch_size)

Epoch 1/10
166/166 ━━━━━━━━━━━━━━━━━━━━ 143s 731ms/step - loss: 0.5137 - sparse_categorical_accuracy: 0.5402
Epoch 2/10
166/166 ━━━━━━━━━━━━━━━━━━━━ 121s 729ms/step - loss: 0.4342 - sparse_categorical_accuracy: 0.5783
Epoch 3/10
166/166 ━━━━━━━━━━━━━━━━━━━━ 121s 729ms/step - loss: 0.4123 - sparse_categorical_accuracy: 0.5927
Epoch 4/10
166/166 ━━━━━━━━━━━━━━━━━━━━ 121s 729ms/step - loss: 0.3994 - sparse_categorical_accuracy: 0.6016
Epoch 5/10
166/166 ━━━━━━━━━━━━━━━━━━━━ 121s 728ms/step - loss: 0.3898 - sparse_categorical_accuracy: 0.6081
Epoch 6/10
166/166 ━━━━━━━━━━━━━━━━━━━━ 121s 729ms/step - loss: 0.3777 - sparse_categorical_accuracy: 0.6162
Epoch 7/10
166/166 ━━━━━━━━━━━━━━━━━━━━ 121s 728ms/step - loss: 0.3622 - sparse_categorical_accuracy: 0.6280
Epoch 8/10
166/166 ━━━━━━━━━━━━━━━━━━━━ 121s 728ms/step - loss: 0.3435 - sparse_categorical_accuracy: 0.6435
Epoch 9/10
166/166 ━━━━━━━━━━━━━━━━━━━━ 121s 728ms/step - loss: 0.3208 - sparse_categorical_accuracy: 0.6625
Epoch 10/10
166/166

# Test the fine-tuned model

## Define the specialized class

In [12]:
class GemmaQA:
    def __init__(self, max_length=512):
        self.max_length = max_length
        self.prompt = "\n\nQuestion:\n{question}\n\nAnswer:\n{answer}"
        self.gemma_lm = gemma_lm
        
    def query(self, question):
        t_start = time()
        response = self.gemma_lm.generate(
            self.prompt.format(
                question=question,
                answer=""), 
            max_length=self.max_length)
        t_end = time()
        display(Markdown(colorize_text(f"{response}\nTotal time: {round(t_end - t_start,2)} sec.")))
        

In [13]:
gemma_qa = GemmaQA()

## Test 1

In [14]:
row = df.iloc[2]
gemma_qa.query(row.question)



**<font color='blue'>Question:</font>**
What is linear regression? When do we use it? 👶

**<font color='redblue'>Answer:</font>**
Linear regression is a statistical method for predicting a single value for a new input, by solving a linear equation with known (or parameterized) constants, or feature values, or regression coefficients.
Total time: 13.74 sec.

## Test 2

In [15]:
row = df.iloc[10]
gemma_qa.query(row.question)



**<font color='blue'>Question:</font>**
What is SGD  —  stochastic gradient descent? What’s the difference with the usual gradient descent? ‍⭐️

**<font color='redblue'>Answer:</font>**
Stochastic gradient descent is a variant of gradient descent where the gradient is calculated using only a random sample from the training set. This means that the update step is only based on a random sample from the training set. The usual gradient descent uses the whole training set to calculate the gradient. This means that the update step is based on all the training examples.

The difference between SGD and gradient descent is that SGD is faster to compute, since it only requires one calculation of the gradient per iteration.
Total time: 2.95 sec.

## Test 3

In [16]:
row = df.iloc[15]
gemma_qa.query(row.question)



**<font color='blue'>Question:</font>**
How to validate your models? 👶

**<font color='redblue'>Answer:</font>**
Answer here
Total time: 0.35 sec.

## Fresh question 1

In [17]:
question = "What is regularization?"
gemma_qa.query(question)



**<font color='blue'>Question:</font>**
What is regularization?

**<font color='redblue'>Answer:</font>**
Regularization is a technique for preventing your neural networks from overfitting. It does this by adding some penalty to your cost function.
Total time: 1.0 sec.

## Fresh question 2

In [18]:
question = "What is SVM?"
gemma_qa.query(question)



**<font color='blue'>Question:</font>**
What is SVM?

**<font color='redblue'>Answer:</font>**
Support Vector Machine (SVM) is a supervised machine learning algorithm that finds a optimal boundary between the data points, called support vectors, such that the data points closest to the boundary are called support vectors.
Total time: 1.38 sec.

## Fresh question 3

In [19]:
question = "What is Dropout?"
gemma_qa.query(question)



**<font color='blue'>Question:</font>**
What is Dropout?

**<font color='redblue'>Answer:</font>**
Dropout is a technique that randomly turns off connections between layers in a neural network, which prevents the network from overfitting.
Total time: 0.95 sec.

## Fresh question 4

In [20]:
question = "Please describe the principle of decision trees."
gemma_qa.query(question)



**<font color='blue'>Question:</font>**
Please describe the principle of decision trees.

**<font color='redblue'>Answer:</font>**
A decision tree is a tree-structured prediction model in which the nodes represent the possible inputs (or features), the branches represent the application of a test on the input, and the leaves (or outcomes) represent the outcomes of the prediction.
Total time: 1.6 sec.

## Fresh question 5

In [21]:
question = "Please describe logistic regression principle."
gemma_qa.query(question)



**<font color='blue'>Question:</font>**
Please describe logistic regression principle.

**<font color='redblue'>Answer:</font>**
Logistic regression is a machine learning algorithm that is used for classification problems. It is also known as the logistic function or sigmoid function. The main idea behind logistic regression is to find a set of weights that minimizes the probability of a prediction being wrong. It is used when we have a binary dependent variable and many independent variables.
Total time: 2.03 sec.

# Conclusions


We fine-tuned Gemma with a set of Data Science interview question and answers.   
Then we tested the model with questions from the dataset used for fine-tuning.  
At the end, we also tested with some free formulated questions, not from the dataset.